In [47]:
'''
In this project I use historical returns data from a porfolio of over 30 indusries in order to identify the optmized 
weights for N portfolio of assets. 
In this part of the Code I create functions necessary for conducting calculations to Minimum Varience Portfolio.  
'''


def portfolio_return(weights, vec_returns):
    '''
    Computes the return of a portfolio. 
    It takes in input a row vector of weights (list of np.array) 
    and a column vector (or pd.Series) of returns
    '''
    return np.dot(weights, vec_returns)

def portfolio_vol(weights, cov_rets):
    '''
    Computes the volatility of a portfolio. 
    It takes in input a vector of weights (np.array or pd.Series) 
    and the covariance matrix of the portfolio asset returns
    '''
    return ( np.dot(weights.T, np.dot(cov_rets, weights)) )**(0.5) 

def annualize_rets(s, periods_per_year):
    '''
    Computes the return per year, or, annualized return.
    The variable periods_per_year can be, e.g., 12, 52, 252, in 
    case of monthly, weekly, and daily data.
    The method takes in input either a DataFrame or a Series and, in the former 
    case, it computes the annualized return for every column (Series) by using pd.aggregate
    '''
    if isinstance(s, pd.DataFrame):
        return s.aggregate( annualize_rets, periods_per_year=periods_per_year )
    elif isinstance(s, pd.Series):
        growth = (1 + s).prod()
        n_period_growth = s.shape[0]
        return growth**(periods_per_year/n_period_growth) - 1

In [9]:
import numpy as np 
import pandas as pd
from scipy.optimize import minimize

%load_ext autoreload
%autoreload 2

df=pd.read_csv(r"/Users/vladimirbarshchuk/Downloads/ind30_m_vw_rets.csv", header = 0, index_col = 0, parse_dates = True)/100
df.index =pd.to_datetime(df.index, format ="%Y%m").to_period("M")
df.columns = df.columns.str.strip()

er = annualize_rets(df["1995":"2000"],12)
cov = df["1995":"2000"].cov()



def minimize_vol(target_return, er, cov):
    n=er.shape[0]
    init_guess = np.repeat(1/n, n)
    bounds =((0.0, 1.0),)*n
    
    weights_sum_to_1 = {'type': 'eq',
                        'fun': lambda weights: np.sum(weights) - 1
    }
    return_is_target = {'type': 'eq',
                        'args': (er,),
                        'fun': lambda weights, er: target_return - portfolio_return(weights,er)
    }
    weights = minimize(portfolio_vol, init_guess,
                       args=(cov,), method='SLSQP',
                       options={'disp': False},
                       constraints=(weights_sum_to_1,return_is_target),
                       bounds=bounds)
    return weights.x

In [52]:
l = ["Games", "Fin", "Paper", "Trans"]
minimize_vol(0.1455, er[l], cov.loc[l,l])

array([0.25143919, 0.21871255, 0.18072117, 0.34912709])

In [53]:

'''
What is the portfolio varience for the weights above ? 

'''

'\nWhat is the portfolio varience for the weights above ? \n\n'

In [54]:
minimum_weights = minimize_vol(0.1455, er[l], cov.loc[l,l])

In [57]:
portfolio_vol(minimum_weights, cov.loc[l,l])

0.04692233208713832